In [65]:
import sys
from pathlib import Path
curr_path = str(Path().absolute()) 
parent_path = str(Path().absolute().parent) 
sys.path.append(parent_path) 

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
n_epochs = 10
path = parent_path+'/mnist'
train_loader = DataLoader(datasets.MNIST(parent_path+'/mnist/', train=True, download=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])), 
                          batch_size=batch_size, 
                          shuffle=True)
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_function = nn.MSELoss().to(device)
log_loss=[]
for epoch in range(n_epochs):
    total_loss = 0
    for data, target in train_loader:
        data,target = data.to(device),target.to(device)
        model.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        log_loss.append(loss)
    total_loss /= len(train_loader.dataset)
    print('{}/{} Loss:'.format(epoch+1, n_epochs), total_loss.item())

[1/10] Loss: 0.008110719732940197
